In [1]:
# Required to load columns with extension types, e.g. json type
import elbow.dtypes
import pandas as pd

from bids2table import bids2table

In [2]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

## Building the index

Generate the BIDS index with 4 parallel workers. Save the index to disk (in parquet format) for easy reload later.

Note that we are simultaneously indexing all datasets in the bids-examples repository.

In [3]:
bids2table(
    root="../bids-examples",
    output="bids-examples.b2t",
    persistent=True,
    overwrite=True,
    workers=4,
    return_table=False,
)

172it [00:00, 406.13it/s, tot=172, good=172, rec=2240, err=0]
193it [00:00, 429.92it/s, tot=193, good=193, rec=2386, err=0]
202it [00:00, 388.18it/s, tot=202, good=202, rec=2828, err=0]
213it [00:00, 409.96it/s, tot=213, good=213, rec=2812, err=0]


With `persistent=True`, the index is saved to disk for later use. By default it's saved to `bids-examples/index.b2t`. The index is saved as a directory of Parquet files, one per worker.

From the [Parquet docs](https://parquet.apache.org/):

> Apache Parquet is an open source, column-oriented data file format designed for efficient data storage and retrieval. It provides efficient data compression and encoding schemes with enhanced performance to handle complex data in bulk.

In [4]:
! ls -lht bids-examples.b2t/

total 1608
-rw-------  1 clane  staff   197K Aug  8 12:38 part-20230808123827-0002-of-0004.parquet
-rw-------  1 clane  staff   240K Aug  8 12:38 part-20230808123827-0003-of-0004.parquet
-rw-------  1 clane  staff   194K Aug  8 12:38 part-20230808123827-0001-of-0004.parquet
-rw-------  1 clane  staff   167K Aug  8 12:38 part-20230808123827-0000-of-0004.parquet


## Load and explore the index

Now when `bids2table` is called again, the persistent index is just loaded.

Each row in the table corresponds to a BIDS data file. The table is organized with several groups of columns:

- dataset (`ds__*`): dataset name, relative dataset path, and the JSON dataset description
- entities (`ent__*`): All [valid BIDS entities](https://bids-specification.readthedocs.io/en/stable/appendices/entities.html) plus an `extra_entities` dict containing any extra entities
- metadata (`meta__*`): BIDS JSON metadata
- file info (`finfo__*`): General file info including the full file path and last modified time

In [5]:
tab = bids2table("../bids-examples", output="bids-examples.b2t")
tab.head(3)

,ds__dataset,ds__dataset_type,ds__dataset_path,ds__dataset_description,ent__sub,ent__ses,ent__sample,ent__task,ent__acq,ent__ce,ent__trc,ent__stain,ent__rec,ent__dir,ent__run,ent__mod,ent__echo,ent__flip,ent__inv,ent__mt,ent__part,ent__proc,ent__hemi,ent__space,ent__split,ent__recording,ent__chunk,ent__atlas,ent__res,ent__den,ent__label,ent__desc,ent__datatype,ent__suffix,ent__ext,ent__extra_entities,meta__json,finfo__file_path,finfo__link_target,finfo__mod_time
0,ds002,None,/Users/clane/Projects/B2T/bids2table/bids-exam...,"{'BIDSVersion': '1.0.0', 'License': 'This data...",15,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,anat,T1w,.nii.gz,{},None,/Users/clane/Projects/B2T/bids2table/bids-exam...,None,1.691420e+09
1,ds002,None,/Users/clane/Projects/B2T/bids2table/bids-exam...,"{'BIDSVersion': '1.0.0', 'License': 'This data...",15,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,anat,inplaneT2,.nii.gz,{},None,/Users/clane/Projects/B2T/bids2table/bids-exam...,None,1.691420e+09
2,ds002,None,/Users/clane/Projects/B2T/bids2table/bids-exam...,"{'BIDSVersion': '1.0.0', 'License': 'This data...",15,None,None,probabilisticclassification,None,None,None,None,None,None,1.0,None,NaN,NaN,NaN,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,func,bold,.nii.gz,{},"{'RepetitionTime': 2.0, 'TaskName': 'probabili...",/Users/clane/Projects/B2T/bids2table/bids-exam...,None,1.691420e+09


In [6]:
print(tab.columns.tolist())

['ds__dataset', 'ds__dataset_type', 'ds__dataset_path', 'ds__dataset_description', 'ent__sub', 'ent__ses', 'ent__sample', 'ent__task', 'ent__acq', 'ent__ce', 'ent__trc', 'ent__stain', 'ent__rec', 'ent__dir', 'ent__run', 'ent__mod', 'ent__echo', 'ent__flip', 'ent__inv', 'ent__mt', 'ent__part', 'ent__proc', 'ent__hemi', 'ent__space', 'ent__split', 'ent__recording', 'ent__chunk', 'ent__atlas', 'ent__res', 'ent__den', 'ent__label', 'ent__desc', 'ent__datatype', 'ent__suffix', 'ent__ext', 'ent__extra_entities', 'meta__json', 'finfo__file_path', 'finfo__link_target', 'finfo__mod_time']


In fact, the dataframe returned by `bids2table` is a special `BIDSTable` subclass of `pandas.DataFrame` with a few extra helper methods.

- You can view the table with [nested columns](https://pandas.pydata.org/docs/user_guide/advanced.html#hierarchical-indexing-multiindex)

In [7]:
tab.nested.head(3)

ds                                                                  \
  dataset dataset_type                                       dataset_path   
0   ds002         None  /Users/clane/Projects/B2T/bids2table/bids-exam...   
1   ds002         None  /Users/clane/Projects/B2T/bids2table/bids-exam...   
2   ds002         None  /Users/clane/Projects/B2T/bids2table/bids-exam...   

                                                     ent               \
                                 dataset_description sub   ses sample   
0  {'BIDSVersion': '1.0.0', 'License': 'This data...  15  None   None   
1  {'BIDSVersion': '1.0.0', 'License': 'This data...  15  None   None   
2  {'BIDSVersion': '1.0.0', 'License': 'This data...  15  None   None   

                                                                               \
                          task   acq    ce   trc stain   rec   dir  run   mod   
0                         None  None  None  None  None  None  None  NaN  None   
1                         None  None  None  None  None  None  None  NaN  None   
2  probabilisticclassification  None  None  None  None  None  None  1.0  None   

                                                                           \
  echo flip inv    mt  part  proc  hemi space split recording chunk atlas   
0  NaN  NaN NaN  None  None  None  None  None   NaN      None   NaN  None   
1  NaN  NaN NaN  None  None  None  None  None   NaN      None   NaN  None   
2  NaN  NaN NaN  None  None  None  None  None   NaN      None   NaN  None   

                                                                       \
    res   den label  desc datatype     suffix      ext extra_entities   
0  None  None  None  None     anat        T1w  .nii.gz             {}   
1  None  None  None  None     anat  inplaneT2  .nii.gz             {}   
2  None  None  None  None     func       bold  .nii.gz             {}   

                                                meta  \
                                                json   
0                                               None   
1                                               None   
2  {'RepetitionTime': 2.0, 'TaskName': 'probabili...   

                                               finfo                            
                                           file_path link_target      mod_time  
0  /Users/clane/Projects/B2T/bids2table/bids-exam...        None  1.691420e+09  
1  /Users/clane/Projects/B2T/bids2table/bids-exam...        None  1.691420e+09  
2  /Users/clane/Projects/B2T/bids2table/bids-exam...        None  1.691420e+09

- You can easily access the dataset (`ds`), entities (`ent`), metadata (`meta`), or file info (`finfo`) subtables.

In [8]:
tab.ent.head(3)

,sub,ses,sample,task,acq,ce,trc,stain,rec,dir,run,mod,echo,flip,inv,mt,part,proc,hemi,space,split,recording,chunk,atlas,res,den,label,desc,datatype,suffix,ext,extra_entities
0,15,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,anat,T1w,.nii.gz,{}
1,15,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,anat,inplaneT2,.nii.gz,{}
2,15,None,None,probabilisticclassification,None,None,None,None,None,None,1.0,None,NaN,NaN,NaN,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,func,bold,.nii.gz,{}


- You can view the full table without the group prefixes

In [9]:
tab.flat.head(3)

,dataset,dataset_type,dataset_path,dataset_description,sub,ses,sample,task,acq,ce,trc,stain,rec,dir,run,mod,echo,flip,inv,mt,part,proc,hemi,space,split,recording,chunk,atlas,res,den,label,desc,datatype,suffix,ext,extra_entities,json,file_path,link_target,mod_time
0,ds002,None,/Users/clane/Projects/B2T/bids2table/bids-exam...,"{'BIDSVersion': '1.0.0', 'License': 'This data...",15,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,anat,T1w,.nii.gz,{},None,/Users/clane/Projects/B2T/bids2table/bids-exam...,None,1.691420e+09
1,ds002,None,/Users/clane/Projects/B2T/bids2table/bids-exam...,"{'BIDSVersion': '1.0.0', 'License': 'This data...",15,None,None,None,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,anat,inplaneT2,.nii.gz,{},None,/Users/clane/Projects/B2T/bids2table/bids-exam...,None,1.691420e+09
2,ds002,None,/Users/clane/Projects/B2T/bids2table/bids-exam...,"{'BIDSVersion': '1.0.0', 'License': 'This data...",15,None,None,probabilisticclassification,None,None,None,None,None,None,1.0,None,NaN,NaN,NaN,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,func,bold,.nii.gz,{},"{'RepetitionTime': 2.0, 'TaskName': 'probabili...",/Users/clane/Projects/B2T/bids2table/bids-exam...,None,1.691420e+09


- You can still slice the table and get back a `BIDSTable`

In [10]:
subtab = tab.iloc[:500]
print(type(subtab), subtab.shape)

<class 'bids2table.table.BIDSTable'> (500, 40)


- You can access flattened JSON metadata. (Nb, normalizing the JSON can be a bit costly. We restrict to the subtable above for convenience.)

In [11]:
subtab.flat_meta.head(3)

,RepetitionTime,TaskName,InstitutionAddress,InstitutionName,InstitutionalDepartmentName,PowerLineFrequency,ManufacturersModelName,EEGReference,Manufacturer,EEGChannelCount,MiscChannelCount,RecordingType,RecordingDuration,SamplingFrequency,EOGChannelCount,ECGChannelCount,EMGChannelCount,SoftwareFilters,onset.Description,onset.Units,duration.Description,duration.Units,trial_type.Description,trial_type.Levels.stimulus,trial_type.Levels.response,response_time.Description,sample.Description,value.Description,value.Levels.2,value.Levels.4,value.Levels.8,value.Levels.16,value.Levels.128,SoftwareVersions,MagneticFieldStrength,ReceiveCoilName,ReceiveCoilActiveElements,ScanningSequence,SequenceVariant,ScanOptions,SequenceName,PulseSequenceDetails,ParallelReductionFactorInPlane,PartialFourier,EchoTime,InversionTime,DwellTime,FlipAngle,MRAcquisitionType,PulseSequenceType,...,FrameDuration,AcquisitionMode,ImageDecayCorrected,ImageDecayCorrectionTime,ReconMatrixSize,ImageVoxelSize,ReconMethodName,ReconMethodParameterLabels,ReconMethodParameterUnits,ReconMethodParameterValues,ReconFilterType,ReconFilterSize,AttenuationCorrection,RawSources,ManufacturerModelName,EstimationPaper,EstimationAlgorithm,EstimationSoftwareName,EstimationSoftwareLanguage,EstimationSoftwareVersion,EstimationSoftwareEnv,MEGChannelCount,MEGREFChannelCount,DewarPosition,EpochLength,ContinuousHeadLocalization,HeadCoilFrequency,MaxMovement,DigitizedLandmarks,DigitizedHeadPoints,AssociatedEmptyRoom,SoftwareFilters.SpatialCompensation.GradientOrder,MEGCoordinateSystem,MEGCoordinateUnits,MEGCoordinateSystemDescription,HeadCoilCoordinateSystem,HeadCoilCoordinateUnits,HeadCoilCoordinateSystemDescription,AnatomicalLandmarkCoordinateSystem,AnatomicalLandmarkCoordinateUnits,AnatomicalLandmarkCoordinateSystemDescription,DigitizedHeadPointsCoordinateSystem,DigitizedHeadPointsCoordinateUnits,DigitizedHeadPointsCoordinateSystemDescription,HeadCoilCoordinates.coil1,HeadCoilCoordinates.coil2,HeadCoilCoordinates.coil3,AnatomicalLandmarkCoordinates.NAS,AnatomicalLandmarkCoordinates.LPA,AnatomicalLandmarkCoordinates.RPA
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,probabilistic classification,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Sorting rows

By default the rows are in arbitrary order. We can sort the rows by dataset, subject, session, task, and run.

In [12]:
sort_tab = tab.sort_entities(["dataset", "sub", "ses", "task", "run"])
sort_tab.head(3)

,ds__dataset,ds__dataset_type,ds__dataset_path,ds__dataset_description,ent__sub,ent__ses,ent__sample,ent__task,ent__acq,ent__ce,ent__trc,ent__stain,ent__rec,ent__dir,ent__run,ent__mod,ent__echo,ent__flip,ent__inv,ent__mt,ent__part,ent__proc,ent__hemi,ent__space,ent__split,ent__recording,ent__chunk,ent__atlas,ent__res,ent__den,ent__label,ent__desc,ent__datatype,ent__suffix,ent__ext,ent__extra_entities,meta__json,finfo__file_path,finfo__link_target,finfo__mod_time
3788,7t_trt,None,/Users/clane/Projects/B2T/bids2table/bids-exam...,"{'BIDSVersion': '1.8.0', 'Name': '7t_trt'}",01,1,None,rest,fullbrain,None,None,None,None,None,1.0,None,NaN,NaN,NaN,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,func,bold,.nii.gz,{},{'CogAtlasID': 'https://www.cognitiveatlas.org...,/Users/clane/Projects/B2T/bids2table/bids-exam...,None,1.691420e+09
3790,7t_trt,None,/Users/clane/Projects/B2T/bids2table/bids-exam...,"{'BIDSVersion': '1.8.0', 'Name': '7t_trt'}",01,1,None,rest,fullbrain,None,None,None,None,None,1.0,None,NaN,NaN,NaN,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,func,physio,.tsv.gz,{},"{'StartTime': 0, 'SamplingFrequency': 100, 'Co...",/Users/clane/Projects/B2T/bids2table/bids-exam...,None,1.691420e+09
3786,7t_trt,None,/Users/clane/Projects/B2T/bids2table/bids-exam...,"{'BIDSVersion': '1.8.0', 'Name': '7t_trt'}",01,1,None,rest,fullbrain,None,None,None,None,None,2.0,None,NaN,NaN,NaN,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,func,bold,.nii.gz,{},{'CogAtlasID': 'https://www.cognitiveatlas.org...,/Users/clane/Projects/B2T/bids2table/bids-exam...,None,1.691420e+09


### Filtering

In addition to all the usual pandas slicing operations, `BIDSTable`s also support higher-level filtering operations inspired by PyBIDS.

In [13]:
filtered = (
    tab
    .filter("task", contains="rest")
    .filter("sub", items=["04", "08"])
    .filter("RepetitionTime", 2.5)
)

filtered

,ds__dataset,ds__dataset_type,ds__dataset_path,ds__dataset_description,ent__sub,ent__ses,ent__sample,ent__task,ent__acq,ent__ce,ent__trc,ent__stain,ent__rec,ent__dir,ent__run,ent__mod,ent__echo,ent__flip,ent__inv,ent__mt,ent__part,ent__proc,ent__hemi,ent__space,ent__split,ent__recording,ent__chunk,ent__atlas,ent__res,ent__den,ent__label,ent__desc,ent__datatype,ent__suffix,ent__ext,ent__extra_entities,meta__json,finfo__file_path,finfo__link_target,finfo__mod_time
1554,synthetic/derivatives/fmriprep,derivative,/Users/clane/Projects/B2T/bids2table/bids-exam...,{'Name': 'fMRIPrep - fMRI PREProcessing workfl...,04,02,None,rest,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,T1w,NaN,None,NaN,None,None,None,None,preproc,func,bold,.nii,{},{'Sources': ['bids:raw:sub-04/ses-02/sub-04_se...,/Users/clane/Projects/B2T/bids2table/bids-exam...,None,1.691420e+09
1567,synthetic/derivatives/fmriprep,derivative,/Users/clane/Projects/B2T/bids2table/bids-exam...,{'Name': 'fMRIPrep - fMRI PREProcessing workfl...,04,02,None,rest,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,MNI152NLin2009cAsym,NaN,None,NaN,None,None,None,None,preproc,func,bold,.nii,{},{'Sources': ['bids:raw:sub-04/ses-02/sub-04_se...,/Users/clane/Projects/B2T/bids2table/bids-exam...,None,1.691420e+09
1576,synthetic/derivatives/fmriprep,derivative,/Users/clane/Projects/B2T/bids2table/bids-exam...,{'Name': 'fMRIPrep - fMRI PREProcessing workfl...,04,01,None,rest,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,T1w,NaN,None,NaN,None,None,None,None,preproc,func,bold,.nii,{},{'Sources': ['bids:raw:sub-04/ses-01/sub-04_se...,/Users/clane/Projects/B2T/bids2table/bids-exam...,None,1.691420e+09
1579,synthetic/derivatives/fmriprep,derivative,/Users/clane/Projects/B2T/bids2table/bids-exam...,{'Name': 'fMRIPrep - fMRI PREProcessing workfl...,04,01,None,rest,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,MNI152NLin2009cAsym,NaN,None,NaN,None,None,None,None,preproc,func,bold,.nii,{},{'Sources': ['bids:raw:sub-04/ses-01/sub-04_se...,/Users/clane/Projects/B2T/bids2table/bids-exam...,None,1.691420e+09
4222,synthetic,raw,/Users/clane/Projects/B2T/bids2table/bids-exam...,{'Name': 'Synthetic dataset for inclusion in B...,04,02,None,rest,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,func,bold,.nii,{},"{'TaskName': 'Rest', 'RepetitionTime': 2.5}",/Users/clane/Projects/B2T/bids2table/bids-exam...,None,1.691420e+09
4235,synthetic,raw,/Users/clane/Projects/B2T/bids2table/bids-exam...,{'Name': 'Synthetic dataset for inclusion in B...,04,01,None,rest,None,None,None,None,None,None,NaN,None,NaN,NaN,NaN,None,None,None,None,None,NaN,None,NaN,None,None,None,None,None,func,bold,.nii,{},"{'TaskName': 'Rest', 'RepetitionTime': 2.5}",/Users/clane/Projects/B2T/bids2table/bids-exam...,None,1.691420e+09


You can also apply multiple filters at the same time.

In [14]:
filtered2 = tab.filter_multi(
    task={"contains": "rest"},
    sub={"items": ["04", "08"]},
    RepetitionTime=2.5,
)

print("Filters equal:", filtered.equals(filtered2))

Filters equal: True


### Getting files

The rows of the table can also be converted to a list of structured `BIDSFile`s.

In [15]:
files = filtered.files

print("First file:", files[0])

First file: BIDSFile(dataset='synthetic/derivatives/fmriprep', root=PosixPath('/Users/clane/Projects/B2T/bids2table/bids-examples/synthetic/derivatives/fmriprep'), path=PosixPath('/Users/clane/Projects/B2T/bids2table/bids-examples/synthetic/derivatives/fmriprep/sub-04/ses-02/func/sub-04_ses-02_task-rest_space-T1w_desc-preproc_bold.nii'), entities=BIDSEntities(sub='04', ses='02', sample=None, task='rest', acq=None, ce=None, trc=None, stain=None, rec=None, dir=None, run=None, mod=None, echo=None, flip=None, inv=None, mt=None, part=None, proc=None, hemi=None, space='T1w', split=None, recording=None, chunk=None, atlas=None, res=None, den=None, label=None, desc='preproc', datatype='func', suffix='bold', ext='.nii', extra_entities={}), metadata={'Sources': ['bids:raw:sub-04/ses-02/sub-04_ses-02_task-rest_bold.nii'], 'TaskName': 'Rest', 'RepetitionTime': 2.5})


In [16]:
print("File paths:\n", "\n".join([str(f.relative_path) for f in files]), sep="")

File paths:
sub-04/ses-02/func/sub-04_ses-02_task-rest_space-T1w_desc-preproc_bold.nii
sub-04/ses-02/func/sub-04_ses-02_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii
sub-04/ses-01/func/sub-04_ses-01_task-rest_space-T1w_desc-preproc_bold.nii
sub-04/ses-01/func/sub-04_ses-01_task-rest_space-MNI152NLin2009cAsym_desc-preproc_bold.nii
sub-04/ses-02/func/sub-04_ses-02_task-rest_bold.nii
sub-04/ses-01/func/sub-04_ses-01_task-rest_bold.nii


## Analyze the table

Next we'll do some more detailed analysis of the table to demonstrate some of the more advanced manipulation that's possible.

### Counting occurrences of BIDS entities

Count the number of non-null entries per BIDS entity.

In [17]:
ent_counts = tab.ent.count(axis=0)
ent_counts

sub               10266
ses                3545
sample               16
task               7972
acq                 422
ce                    0
trc                   0
stain                 8
rec                  58
dir                   3
run                6736
mod                   1
echo                541
flip                 53
inv                  20
mt                   25
part                 16
proc                208
hemi                 83
space               310
split                 0
recording             5
chunk                 8
atlas                 0
res                  84
den                   0
label                84
desc                280
datatype           8885
suffix            10218
ext               10266
extra_entities    10266
dtype: int64

We see that some BIDS entities never appear in any of the example datasets.

In [18]:
ent_counts[ent_counts == 0]

ce       0
trc      0
split    0
atlas    0
den      0
dtype: int64

### File counts

Count the number of data files per dataset and the number of files with json metadata.

In [19]:
tab.flat.groupby("dataset").agg(
    {"file_path": "count", "json": "count"}
)

,file_path,json
dataset,,
7t_trt,635,350
asl001,4,2
asl002,5,3
asl003,5,3
asl004,6,4
asl005,5,3
ds000001-fmriprep,416,52
ds000117,1105,657
ds000246,33,23


## Using the command-line interface

bids2table comes with a command-line iterface `bids2table` (alias `b2t`). Check out the help message.

In [20]:
! bids2table -h

usage: bids2table [-h] [--output OUTPUT] [--incremental] [--overwrite]
                  [--workers COUNT] [--worker_id RANK] [--verbose]
                  ROOT

positional arguments:
  ROOT                  Path to BIDS dataset

options:
  -h, --help            show this help message and exit
  --output OUTPUT, -o OUTPUT
                        Path to output parquet dataset directory (default:
                        {ROOT}/index.b2t)
  --incremental, --inc  Update index incrementally with only new or changed
                        files.
  --overwrite, -x       Overwrite previous index.
  --workers COUNT, -w COUNT
                        Number of worker processes. Setting to -1 runs as many
                        processes as there are cores available. (default: 1)
  --worker_id RANK, --id RANK
                        Optional worker ID to use when scheduling parallel
                        tasks externally. Incompatible with --overwrite.
                        (default: None)


Re-generate the index using the CLI.

In [21]:
! bids2table -o bids-examples.b2t -x -w 4 ../bids-examples/

172it [00:00, 400.87it/s, tot=172, good=172, rec=2240, err=0]
193it [00:00, 426.44it/s, tot=193, good=193, rec=2386, err=0]
213it [00:00, 402.36it/s, tot=213, good=213, rec=2812, err=0]
202it [00:00, 381.92it/s, tot=202, good=202, rec=2828, err=0]


You can also generate each partition independently by calling `bids2table` with `--worker_id`. This can be useful in HPC environments where you want to schedule many extraction tasks in parallel through a scheduler like [SLURM](https://slurm.schedmd.com/documentation.html).


```bash
# Can't use --overwrite together with --worker_id
# Remove in advance
rm -r bids-examples.b2t

for worker_id in {0..3}; do
  bids2table -o bids-examples.b2t --worker_id $worker_id --workers 4 ../bids-examples/ &
done
```